In [18]:
from elasticsearch import Elasticsearch
from elasticsearch_llm_cache import ElasticsearchLLMCache
from elasticsearch.exceptions import NotFoundError

# common libraries
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic

# load our environment file
load_dotenv()

es_url = f"https://{os.environ['elasticsearch_user']}:{os.environ['elasticsearch_pw']}@{os.environ['elasticsearch_host']}:{os.environ['elasticsearch_port']}"
# es_index= os.environ['elasticsearch_index']
os.environ['OPENAI_API_KEY'] = os.environ['openai_api_key']
open_api_key=os.environ['OPENAI_API_KEY']

# define our API Key
openai.api_key = os.getenv("openai_api_key")

es = Elasticsearch([es_url])

index_name = 'cache'
model_id = '.elser_model_1'

if es.ml.get_trained_models(model_id=model_id):
    ic(f'{model_id} exists.')
else:
    ic(f'{model_id} does not exist.')

if es.indices.exists(index=index_name):
    ic(f'{index_name} exists.')
else:
    ic(f'{index_name} does not exist.')
    es.indices.create(index= index_name)


INFO:elastic_transport.transport:GET https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/.elser_model_1 [status:200 duration:0.118s]
ic| f'{model_id} exists.': '.elser_model_1 exists.'
INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/cache [status:200 duration:0.036s]
ic| f'{index_name} exists.': 'cache exists.'


In [9]:
# Initialize Elasticsearch LLM Cache
llm_cache = ElasticsearchLLMCache(es)

# Query the cache
cache_response = llm_cache.query(prompt_text="Hello, how can I help?")

# If no cache hit, add new response to cache
if not cache_response:
    llm_response = "I'm here to assist you!"  # Assume this response is fetched from LLM
    llm_cache.add(prompt="Hello, how can I help?", response=llm_response)

INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache [status:404 duration:0.045s]
INFO:elastic_transport.transport:PUT https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache [status:200 duration:0.129s]
INFO:elasticsearch_llm_cache:Index llm_cache created.
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache/_search [status:404 duration:0.060s]


NotFoundError: NotFoundError(404, 'resource_not_found_exception', 'Could not find trained model [sentence-transformers__all-distilroberta-v1]')